In [14]:
import openseespy.opensees as osp
import numpy as np
from dataset.generation.generator import StructuralDatasetGenerator

In [42]:
osp.wipe()

# set modelbuilder
osp.model('basic', '-ndm', 2, '-ndf', 2)

osp.node(0, 0., 0.)
osp.node(1, 0., 2.)
osp.node(2, 2., 0.)
osp.node(3, 2., 2.)
osp.node(4, 4., 0.)

osp.fix(0, 1, 1)
osp.fix(1, 1, 1)
osp.fix(4, 1, 0)

osp.uniaxialMaterial("Elastic", 1, 150.0e6)

osp.element("Truss", 0, 0, 2, 1.0, 1)
osp.element("Truss", 1, 2, 4, 1.0, 1)
osp.element("Truss", 2, 1, 3, 1.0, 1)
osp.element("Truss", 3, 2, 3, 1.0, 1)
osp.element("Truss", 4, 1, 2, 1.0, 1)
osp.element("Truss", 5, 3, 4, 1.0, 1)

# create TimeSeries
osp.timeSeries("Linear", 1)

# create a plain load pattern
osp.pattern("Plain", 1, 1)

osp.load(3, -30.0e3, 0.0)
osp.load(4, 0.0, 50.0e3)

# create SOE
osp.system("BandSPD")

# create DOF number
osp.numberer("RCM")

# create constraint handler
osp.constraints("Plain")

# create integrator
osp.integrator("LoadControl", 1.0)

# create algorithm
osp.algorithm("Linear")

# create analysis object
osp.analysis("Static")

# perform the analysis
osp.analyze(1)

0

In [63]:
def _get_r(a):
    '''Compute member rotation matrix'''
    c = np.cos(a)
    s = np.sin(a)
    return np.array([[c, s, 0, 0],
                     [-s, c, 0, 0],
                     [0, 0, c, s],
                     [0, 0, -s, c]])

def _get_k_loc(idx):
    '''Compute the local element matrix'''
    return osp.basicStiffness(idx) * np.array([[1, 0, -1, 0],
                                               [0, 0, 0, 0],
                                               [-1, 0, 1, 0],
                                               [0, 0, 0, 0]])

def _get_k_global(k_loc, angle):
    '''Compute the global element matrix'''
    r = _get_r(angle)
    return r.T @ k_loc @ r

In [64]:
def get_K(ndof=2):
    '''Compute the global structure matrix'''
    # Parameters
    nodes = np.array([osp.nodeCoord(i) for i in osp.getNodeTags()], dtype=int)
    elems = np.array([osp.eleNodes(i) for i in osp.getEleTags()])

    elems_vec = np.array([nodes[e] - nodes[s] for s, e in elems])
    elems_angle = np.array([np.arctan2(*v[::-1]) - np.arctan2(0, 1) for v in elems_vec])

    # Stiffness matrix
    K = np.zeros((ndof * len(osp.getNodeTags()), ndof * len(osp.getNodeTags())))
    for idx in range(len(elems)):
        # Get element stiffness matrix
        s_i, e_i = elems[idx] * ndof
        angle = elems_angle[idx]

        k_loc = _get_k_loc(idx)
        k_glob = _get_k_global(k_loc, angle)

        # Assemble global stiffness matrix
        K[s_i: s_i + ndof, s_i: s_i + ndof] += k_glob[0:2, 0:2]
        K[e_i: e_i + ndof, e_i: e_i + ndof] += k_glob[2:4, 2:4]
        K[s_i: s_i + ndof, e_i: e_i + ndof] += k_glob[0:2, 2:4]
        K[e_i: e_i + ndof, s_i: s_i + ndof] += k_glob[2:4, 0:2]

    # Boundary condition
    for idx in range(len(nodes)):
        for i in osp.getFixedDOFs(idx):
            dof = ndof * idx + i - 1  # OSP indices starts at 1

            K[dof, :] = 0.
            K[:, dof] = 0.
            K[dof, dof] = 1.

    return K

In [65]:
k = get_K()
u = np.reshape([u for i in range(5) for u in osp.nodeDisp(i)], (10,1))

In [66]:
1000*u

array([[         0.],
       [         0.],
       [         0.],
       [         0.],
       [   3.33e-01],
       [   2.22e+00],
       [  -1.07e+00],
       [   2.89e+00],
       [         0.],
       [   5.84e+00]])

In [71]:
np.round(k@u)

array([[         0.],
       [         0.],
       [         0.],
       [         0.],
       [         0.],
       [         0.],
       [  -3.00e+04],
       [         0.],
       [         0.],
       [   5.00e+04]])

In [68]:
print(get_K() / 75e6)

[[        ~0.          0.          0.          0.          0.          0.          0.          0.          0.          0.]
 [         0.         ~0.          0.          0.          0.          0.          0.          0.          0.          0.]
 [         0.          0.         ~0.          0.          0.          0.          0.          0.          0.          0.]
 [         0.          0.          0.         ~0.          0.          0.          0.          0.          0.          0.]
 [         0.          0.          0.          0.    2.35e+00   -3.54e-01         ~0.         ~0.          0.          0.]
 [         0.          0.          0.          0.   -3.54e-01    1.35e+00         ~0.   -1.00e+00          0.          0.]
 [         0.          0.          0.          0.         ~0.         ~0.    1.35e+00   -3.54e-01          0.    3.54e-01]
 [         0.          0.          0.          0.         ~0.   -1.00e+00   -3.54e-01    1.35e+00          0.   -3.54e-01]
 [         0.   

In [34]:
for i in range(5):
    print(osp.nodeDisp(i))

[0.0, 0.0]
[0.0, 0.0]
[-0.00033333333333333327, 0.00221895141649746]
[0.00026666666666666625, 0.0028856180831641266]
[0.0, 0.00503790283299492]
